In [ ]:
from IPython.core.display import HTML
HTML("""
<style>

div.cell { /* Tunes the space between cells */
margin-top:1em;
margin-bottom:1em;
}

div.text_cell_render h1 { /* Main titles bigger, centered */
font-size: 2.2em;
line-height:1.4em;
text-align:center;
}

div.text_cell_render h2 { /*  Parts names nearer from text */
margin-bottom: -0.4em;
}


div.text_cell_render { /* Customize text cells */
font-family: 'Times New Roman';
font-size:1.5em;
line-height:1.4em;
padding-left:3em;
padding-right:3em;
}
</style>
""")

In [ ]:
import numpy as np
from numpy import random
import IPython
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)

import sys
def progress_bar(percent, size = 20, message=''):
    percent = int(percent)
    fill = int(percent * size/100)
    sys.stdout.write('\r[' + 'X'*fill + ' '*(size-fill) + '] {:3d}% complete. '.format(percent) + message)
    sys.stdout.flush

# Rock Paper Scissors

## Learning how to beat a predictable opponent

A strategy in rock-paper-scissors consists of a probability distribution in which a player chooses their move. Rock paper scissors has a payout matrix given by

$$M_{ij} = \begin{matrix} &\begin{matrix}\text{rock}&\text{paper}&\text{scissors}\end{matrix}\\ \begin{matrix}\text{rock} \\ \text{paper} \\ \text{scissors} \end{matrix} & \begin{pmatrix}\quad \pm0 \quad&\quad -1\quad &\quad +1\quad \\+1 & \pm0 & -1 \\-1 & +1 & \pm0 \end{pmatrix}\end{matrix}$$.

The rows represent player 1's moves while the columns represent player 2's moves. Once both players choose their move, player 1 is rewarded with the corresponding value in the payout matrix and player 2 is rewarded with the negative of player 1's reward (zero-sum). Player 2's payout matrix is $- M_{ji}$ the negative transpose of player 1's, but since the payout matrix is skew-symmetric $M_{ij} = - M_{ji}$, the payout matrices are the same (symmetric).

In [ ]:
def play(player1,player2):

    r1=random.random()
    for i,pi in enumerate(player1):
        r1 -= pi
        if r1 < 0:
            move1 = i
            break

    r2=random.random()
    for j,pj in enumerate(player2):
        r2 -= pj
        if r2 < 0:
            move2 = j
            break

    return move1,move2
TEMP = 0.25
def softmax(ar):
    ar = np.copy(ar)
#    print ar
    ar -= np.max(ar)
    exp = np.exp(ar / TEMP)
    return exp / np.sum(exp)

def learn(player1,player2,payout,learning_rate=0.001, learn = [True,True],duration = 1000,print_freq = 100):
    player1 = np.copy(player1)
    player2 = np.copy(player2)
    #learning_rate = 0.001
    decay = 1 # 0.9995
    p1_options = len(player1)
    p2_options = len(player2)
    history = [np.copy(player1)]
    score = 0
    cum_score = [0]
    f,_= plt.subplots(1,3, sharey=True,figsize = (15,5))
    ax1 = plt.subplot(131, aspect='equal', adjustable='datalim')
    ax2 = plt.subplot(132, aspect = 1,adjustable='datalim')
    ax3 = plt.subplot(133, aspect=1, adjustable='datalim')
    for session in range(duration):
        move1,move2 = play(softmax(player1),softmax(player2))
        player1 *= decay
        player2 *= decay
        #print move1,move2,player1
        if learn[0]:
            player1[move1] = learning_rate*payout[move1,move2] + (1-learning_rate)*player1[move1]

#             if np.sum(np.abs(player1)) <1:
#                 player1 += np.ones(p1_options)*1.0/p1_options * (1 - np.sum(np.abs(player1)))
                
                
        if learn[1]:
            player2[move2] = learning_rate*payout[move2,move1] + (1-learning_rate)*player2[move2]

#             if np.sum(np.abs(player2)) <1:
#                 player2 += np.ones(p2_options)*1.0/p2_options * (1-np.sum(np.abs(player2)))

        score += payout[move1,move2]
        cum_score.append(score)

        #print player1

        #negativity control
#         player1 = player1 * (player1>0)
#         player2 = player2 * (player2>0)

        #renormalize
#         player1 = player1 / float(np.sum(np.abs(player1))) 
#         player2 = player2 / float(np.sum(np.abs(player2)))


        if print_freq and session%print_freq==0:
            
            ax1.cla()
            ax2.cla()
            ax3.cla()
            ax1.bar(np.arange(p1_options) + 0.5 , softmax(player1))
            ax1.set_ylim(0,1)
            
            ax2.plot(cum_score , linewidth = 3)
            
            
            ax3.bar(np.arange(p2_options) + 0.5 , softmax(player2))
            ax3.set_ylim(0,1)

            IPython.display.clear_output(wait=True)
            IPython.display.display(plt.gcf())
            
        if not print_freq and session%1000==0:
            #progress_bar(100.0 * session / duration , 50, "current strategy: Rock:{:.3f}, Paper:{:.3f}, Scissors:{:.3f}".format(
            #    *player1))
            fill = map(int,list(softmax(player1)*100))
            sys.stdout.write(("\033[2A\r    Rock : [" + 'X'*fill[0] + ' '*(100-fill[0]) + '] ({:.3f})\n' + 
                                       "   Paper : [" + 'X'*fill[1] + ' '*(100-fill[1]) + '] ({:.3f})\n' + 
                                       "Scissors : [" + 'X'*fill[2] + ' '*(100-fill[2]) + '] ({:.3f})').format(*softmax(player1)))
            IPython.display.clear_output(wait = True)
    return player1


In [ ]:
payout = np.array([[ 0,-1, 1],
                   [ 1, 0,-1],
                   [-1, 1, 0]])
player1 = np.array([0.0,1.0,0.0]).astype(float)
player2 = np.array([0.0,0.0,1.0]).astype(float)

In this example, player 2 will only be playing scissors, and player 1 will be learning how to beat player 2's strategy

Let's see if player 1 can learn the optimal strategy to beat player 2. Player 1 will begin with the worst possible strategy, choosing paper every time. With the initial strategies, player 2 wins every game.

In [ ]:
score = 0
for _ in range(10000):
    score += payout[play(player1,player2)]
print "player 1's score is : {}".format(score)

In [ ]:
learn(player1,player2,payout,learn = [True,False],learning_rate = 0.003,duration = 2000,print_freq = 100)

Great! But what if both players try to learn???

In [ ]:
learn(player1,player2,payout,learning_rate = 0.01,duration = 2000, print_freq = 25)

We can actually get very good results if we just stop plotting, plotting is very expensive.

In [ ]:
learn(player1,player2,payout,learning_rate = 0.0001,duration = 300000, print_freq = 0)

Bonus question: what happens if we modify the payout matrix? What is the optimal strategy?

In [ ]:
payout = np.array([[ 0,-2, 1],
                   [ 2, 0,-1],
                   [-1, 1, 0]])
#You get twice the payout for winning with paper

softmax(learn(player1,player2,payout,learning_rate = 0.00001,duration = 1000000,print_freq= 0 ))